In [5]:
from DeepGATE import Explorer
import numpy as np

## Exemple d'utilisation

In [ ]:
#model_path = 'Results/newHS37_exon_start/best_metrics_model.hdf5'
model_path = '../DeepGAP/Results/HS37_coverage_BN_3/best_metrics_model.hdf5'

# Chargement du modèle et des poids
exemple = Explorer(model_path)

# Chargement de l'input (en one_hot)
data = np.load(f'../Data/DNA/HS38/one_hot/chr1.npy').astype('int8')
